In [1]:
import json
import sqlalchemy
import pandas as pd
import boto3
import time
import numpy as np

In [2]:
pd.options.display.max_columns = 999

In [3]:
config = json.load(open('../config/config.json', 'r'))
db = config['db']
conn_template = 'mysql+pymysql://{}:{}@{}:{}/{}'
connstr = conn_template.format(db['username'], db['password'], db['host'], db['port'], db['dbname'])
e = sqlalchemy.create_engine(connstr)

In [4]:
query = """
select t2.*, t1.address_line1, t1.shipping_city, t1.shipping_state, t1.shipping_postal_code, t1.shipping_country
from paypal_payments t1
join (
	select email, first_name, lastname
	 , max(create_time) as latest
	 , min(create_time) as since
	 , max(total) as total
	 , max(id) as id
	from paypal_payments
	group by email, first_name, lastname
) t2
on t1.id = t2.id
"""
pp = pd.read_sql(query, e)

In [7]:
query = """
SELECT *
FROM stripe_subscribers
"""
st = pd.read_sql(query, e)

In [8]:
import stripe
config = json.load(open('../config/config.json', 'r'))
db = config['db']
conn_template = 'mysql+pymysql://{}:{}@{}:{}/{}'
connstr = conn_template.format(db['username'], db['password'], db['host'], db['port'], db['dbname'])
e = sqlalchemy.create_engine(connstr)
stripe.api_key = config['stripe_id']

In [9]:
client = boto3.client('dynamodb')

In [10]:
response = client.scan(TableName='orders')

In [11]:
items = response['Items']

In [12]:
customers = {}
for item in items:
    customer = item['customer']['M']
    line1 = customer['street_1']['S']
    if 'street_2' in customer:
        line2 = customer['street_2']['S']
    else:
        line2 = ''
    city = customer['city']['S']
    state = customer['state']['S']
    first_name = customer['first_name']['S']
    last_name = customer['last_name']['S']
    email = customer['email']['S']
    postal = customer['zip']['S']
    c = {'line1': line1, 'line2': line2, 'city': city, 'state': state, 'first_name': first_name, 'last_name': last_name, 'email': email, 'postal': postal}
    customers[email] = c

In [13]:
cols = ['line1', 'line2', 'city', 'state', 'first_name', 'last_name', 'email', 'postal', 'first_name', 'last_name']
for r in range(st.shape[0]):
    row = st.iloc[r]
    idx = st.index[r]
    email = row['email']
    if email in customers:
        customer = customers[email]
        for col in cols:
            value = customer[col]
            st.set_value(idx, col, value, takeable=False)
    else:
        print("Could not find " + email)

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


In [14]:
# TODO: update Stripe customers with this data

In [15]:
st.rename(columns={'line1':'address_line1','line2':'address_line2', 'city':'shipping_city', 'state':'shipping_state', 'postal':'shipping_postal_code', 'country':'shipping_country'}, inplace=True)

In [16]:
pp.rename(columns={'lastname':'last_name'}, inplace=True)

In [17]:
pp['address_line2'] = ''

In [18]:
cols = ['first_name', 'last_name', 'email', 'address_line1', 'address_line2', 'shipping_city', 'shipping_state', 'shipping_postal_code', 'shipping_country']

In [19]:
df = pd.concat([pp[cols], st[cols]])
df.index = np.arange(df.shape[0])

In [20]:
df['country'] = df['shipping_country'].apply(lambda x: '' if x=='US' else x)

In [22]:
df.to_csv('membership.csv', sep='\t')

In [36]:
formatted = []
for r in range(df.shape[0]):
    row = df.iloc[r]
    first_name = row['first_name']
    last_name = row['last_name']
    address_line1 = row['address_line1']
    shipping_city = row['shipping_city']
    shipping_state = row['shipping_state']
    shipping_postal_code = row['shipping_postal_code']
    country = row['country']
    tpl = """<td>{first_name} {last_name}<br/>
{address_line1}<br/>
{shipping_city}, {shipping_state} {shipping_postal_code}<br/>
{country}</td>
""".format(first_name=first_name, last_name=last_name, address_line1=address_line1, shipping_city=shipping_city, shipping_state=shipping_state, shipping_postal_code=shipping_postal_code, country=country, email=email)
    formatted.append(tpl)

i = 0
bdy = "<table>"
while i < len(formatted):
    tpl = formatted[i]
    if i % 1 == 0:
        if i > 0:
            bdy += "</tr>"
        bdy += "<tr>"        
    bdy += tpl
    if i % 3 != 2:
        bdy += "<td><td/>"
    i += 1

if i % 3 != 1:
    bdy += "</tr>"
bdy += "</table>"

f = open('table2.htm', 'w')
f.write(bdy)
f.close()

In [106]:
tpl = """
<p>Hi {first_name},</p>

<p>We appreciate for your support of Data Skeptic.  We're planning to send something small to you via the US Postal Service.  Below is the address we intend to use.  If this is wrong in any way or will not reach you, simply reply by Dec 20th and we'll get our records updated.</p>

<p>If the address below is correct, and you wouldn't mind a quick confirmation reply, we'd appreciate that too!</p>

<p><b>Address on file:</b></p>
{first_name} {last_name}<br/>
{address_line1}<br/>
{shipping_city}, {shipping_state} {shipping_postal_code}<br/>
{country}<br/>

<p>Thanks!</p>

<p>Kyle & the Data Skeptic team</p>
"""

In [118]:
responses = {}

In [113]:
client = boto3.client('ses')

In [ ]:
CHARSET = "UTF-8"

for r in range(df.shape[0]):
    row = df.iloc[r]
    first_name = row['first_name']
    last_name = row['last_name']
    address_line1 = row['address_line1']
    shipping_city = row['shipping_city']
    shipping_state = row['shipping_state']
    shipping_postal_code = row['shipping_postal_code']
    country = row['country']
    bdy = tpl.format(first_name=first_name, last_name=last_name, address_line1=address_line1, shipping_city=shipping_city, shipping_state=shipping_state, shipping_postal_code=shipping_postal_code, country=country)
    to = row['email']
    if to not in responses:        
        print(to, first_name, last_name, bdy[0:100])
        response = client.send_email(
            Destination={
                'ToAddresses': [
                    to,
                ],
            },
            Message={
                'Body': {
                    'Html': {
                        'Charset': CHARSET,
                        'Data': bdy,
                    }
                },
                'Subject': {
                    'Charset': CHARSET,
                    'Data': 'Data Skeptic mailing address confirmation',
                },
            },
            Source='kyle@dataskeptic.com'
        )
        responses[to] = response
        time.sleep(1)


In [ ]:
for r in range(df.shape[0]):
    row = df.iloc[r]
    first_name = row['first_name']
    last_name = row['last_name']
    address_line1 = row['address_line1']
    shipping_city = row['shipping_city']
    shipping_state = row['shipping_state']
    shipping_postal_code = row['shipping_postal_code']
    country = row['country']
    email = row['email']
    tpl = """{first_name} {last_name}
{address_line1}
{shipping_city}, {shipping_state} {shipping_postal_code}
{country}
{email}
""".format(first_name=first_name, last_name=last_name, address_line1=address_line1, shipping_city=shipping_city, shipping_state=shipping_state, shipping_postal_code=shipping_postal_code, country=country, email=email)
    #print(tpl)
    #print("---------------------------------")